每一個迭代都是一個版本的模型，並非只能保存最終結果，依據過程中觀察可選擇效能較佳模型做為生成環境應用。這部份TensorFlow 2.0提供了`tf.train.Checkpoint`，也保留keras內的callback function。來看看怎麼處理。

不同於保存完整模型，Checkpoint只保留參數，沒有模型架構，因此如果整個模型帶著走的話就需要另外使用SavedModel，這後續會說明。

In [2]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.0.0'

指定硬體資源，相關可[參考](https://hackmd.io/@shaoeChen/ryWIV4vkL)

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')

In [4]:
gpus 

[PhysicalDevice(name=u'/physical_device:GPU:0', device_type=u'GPU'),
 PhysicalDevice(name=u'/physical_device:GPU:1', device_type=u'GPU')]

In [5]:
tf.config.experimental.set_visible_devices(devices=gpus[1], device_type='GPU')

In [6]:
tf.config.experimental.set_memory_growth(device=gpus[1], enable=True)

原本的class就不要用了，讓它隨風去，一樣取得MNIST資料集並做標準化。

In [7]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.expand_dims(x_train / 255., -1)
x_test = np.expand_dims(x_test / 255., -1)

In [8]:
x_train.shape, x_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

將資料集與標籤做為參數提供給`tf.data`

In [9]:
datasets = tf.data.Dataset.from_tensor_slices((x_train, y_train))

In [10]:
datasets

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float64, tf.uint8)>

In [11]:
datasets = datasets.shuffle(buffer_size=128, seed=10).batch(128).repeat()

利用標準的keras Sequential來建置模型

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='tanh'),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax'),
])


確認模型

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               30840     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

編譯模型

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

訓練之前先定義保存模型的方法

首先使用`tf.train.Checkpoint`，裡面的鍵值是我們可以自定義的

In [15]:
checkpoint = tf.train.Checkpoint(model=model)

也可以這麼定義

In [16]:
checkpoint = tf.train.Checkpoint(mymodel = model)

上面的`mymodel`就是自定義的鍵值

訓練模型

In [17]:
%%time
model.fit(datasets,
          epochs=5, 
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 5s 10ms/step - loss: 1.0049 - accuracy: 0.7659
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.3389 - accuracy: 0.9083
Epoch 3/5
468/468 [==============================] - 2s 5ms/step - loss: 0.2341 - accuracy: 0.9338
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1806 - accuracy: 0.9486
Epoch 5/5
468/468 [==============================] - 3s 6ms/step - loss: 0.1473 - accuracy: 0.9582
CPU times: user 18.5 s, sys: 2.03 s, total: 20.5 s
Wall time: 15.1 s


接下來利用`checkpoing.save`來保存模型，參數為欲保存模型的路徑

In [18]:
checkpoint.save('./model.ckpt')

'./model.ckpt-1'

執行之後在相關路徑就會看到多了四個檔案，這個可以多次保存，每次的保存都會產生新的檔案

In [19]:
checkpoint.save('./model.ckpt')

'./model.ckpt-2'

透過`tf.train.latest_checkpoint`可以快速瞭解目前最新的版本是那一個

In [20]:
tf.train.latest_checkpoint('./')

u'./model.ckpt-2'

模型保存之後就會在需要使用的地方還原它，步驟很簡單，只是要注意的一點是，你的鍵值必須與當初設置保存的鍵值一致。

下面設置一個架構一樣的模型

In [21]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='valid', activation='tanh'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='tanh'),
    tf.keras.layers.Dense(84, activation='tanh'),
    tf.keras.layers.Dense(10, activation='softmax'),
])


故意使用錯誤的鍵值來觀察

In [22]:
ckeckpoint1 = tf.train.Checkpoint(abc=model1)

In [23]:
ckeckpoint1.restore(tf.train.latest_checkpoint('./'))

這是以訓練好的模型來預測測試資料集並只顯示第一筆資料

In [27]:
model.predict(x_test)[0]

array([7.64146534e-05, 1.44986352e-05, 2.80535198e-04, 8.07540026e-04,
       1.06578555e-05, 1.74065382e-04, 1.84227630e-07, 9.98010457e-01,
       1.65108013e-05, 6.09055278e-04], dtype=float32)

這是鍵值錯誤情況下還原並預測測試資料集，可以發現所得結果是不一樣的

In [28]:
model1.predict(x_test)[0]

array([0.11440124, 0.07752153, 0.09761439, 0.09649299, 0.11129649,
       0.13053681, 0.08337555, 0.09084191, 0.08159096, 0.11632818],
      dtype=float32)

正確還原鍵值之後再測試一次

In [29]:
ckeckpoint1 = tf.train.Checkpoint(mymodel=model1)

In [30]:
ckeckpoint1.restore(tf.train.latest_checkpoint('./'))

很清楚的發現，所得的結果與訓練模型一致

In [31]:
model1.predict(x_test)[0]

array([7.64146534e-05, 1.44986352e-05, 2.80535198e-04, 8.07540026e-04,
       1.06578555e-05, 1.74065382e-04, 1.84227630e-07, 9.98010457e-01,
       1.65108013e-05, 6.09055278e-04], dtype=float32)

透過這個案例發現到一件恐怖的事情，就是錯誤的鍵值並不會有異常的產生，所使用的時候記得再三確認。

`tf.train.Checkpoint`的優點在於，在調整`checkpoint.restore()`的時候，模型的變數尚未被建立的情況下，Checkpoint會等到變數被建立的時候再還原數值。

你可以在迭代過程中自己設置每幾個epoch就自動保存一次

迭代過多可能留下過多不必要的模型，如果只需要最新幾個的話，那可以透過`tf.train.CheckpointManager`來設置，在new一個`tf.train.Checkpoint`之後再執行即可

In [32]:
checkpoint2 = tf.train.Checkpoint(mymodel=model)
check_manager2 = tf.train.CheckpointManager(checkpoint2, directory='./model', checkpoint_name='model.ckpt', max_to_keep=5)

上面的設置代表我們只保留最多5個模型記錄

原始keras的api也提供callback可以在訓練過程中保存模型，非常簡潔又便利

In [36]:
callback_checkpoing = tf.keras.callbacks.ModelCheckpoint(filepath='./model.hdf5', 
                                                         monitor='loss',
                                                         verbose=0,
                                                         save_best_only=True,
                                                         mode='auto')

上面只是簡單一個callback function，監控loss，在loss有優化的情況下才保留(因為設置`save_best_only=True`

接下來只需要在`model.fit`中設置這個callback function即可

In [37]:
%%time
model.fit(datasets,
          epochs=5, 
          callbacks=[callback_checkpoing],
          steps_per_epoch=int(len(x_train)/128))

Train for 468 steps
Epoch 1/5
468/468 [==============================] - 3s 6ms/step - loss: 0.0728 - accuracy: 0.9785
Epoch 2/5
468/468 [==============================] - 3s 6ms/step - loss: 0.0673 - accuracy: 0.9803
Epoch 3/5
468/468 [==============================] - 3s 6ms/step - loss: 0.0629 - accuracy: 0.9815
Epoch 4/5
468/468 [==============================] - 3s 6ms/step - loss: 0.0588 - accuracy: 0.9826
Epoch 5/5
468/468 [==============================] - 3s 6ms/step - loss: 0.0558 - accuracy: 0.9836
CPU times: user 17.8 s, sys: 1.78 s, total: 19.6 s
Wall time: 13.8 s


上面程式碼執行之後就會產出一個`model.hdf5`的檔案，當然你可以定義使用`epoch`做為檔案名稱，方便觀察記錄。

In [3]:
callback_checkpoing = tf.keras.callbacks.ModelCheckpoint(filepath='./model-{epoch:02d}.hdf5', 
                                                         monitor='loss',
                                                         verbose=0,
                                                         save_best_only=True,
                                                         mode='auto')